# CZ4041 Group 28 Notebook 2
This Notebook Uses the Model Trained in Notebook 1 and Applies it to the Train Data Set. 

This Notebook Concludes with a Generated .csv file Indicating allMissing Score of the Train Dataset Passed through the Model.

To Run this Notebook, Import the .h5 File from the Notebook 1<br>
The .h5 File is:

- model.h5

In [ ]:
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

import keras
import tensorflow as tf
from keras import backend as K
from keras import layers
from keras.applications import DenseNet121
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
from keras.models import Sequential
from keras.optimizers import Adam, Nadam
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

## Using Trained Model on Train Set
We will attempt to get the csv file for the train set. The csv file shall contain the allMissing scores for all the train images.

In [ ]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
os.listdir('../input/stage1/')

In [ ]:
from keras.models import load_model

dependencies={ "recall_m": recall_m,"precision_m": precision_m,"f1_m": f1_m}
model = load_model("../input/stage1/model.h5",
                   custom_objects = dependencies
                  )

In [ ]:
train_df = pd.read_csv('../input/severstal-steel-defect-detection/train.csv').sort_values(by=['ImageId'])

In [ ]:
train_folder = os.path.join('/kaggle/input','/severstal-steel-defect-detection/train_images')

train_images_set = set()
for dirname, _, filenames in os.walk('/kaggle/input/severstal-steel-defect-detection'):
    for filename in filenames:
        if dirname == '/kaggle/input/severstal-steel-defect-detection/train_images':
            train_images_set.add(filename)
    if dirname == '/kaggle/input/severstal-steel-defect-detection/train_images':
        print(dirname)
        print("No_Train_Images: " + str(len(train_images_set)))

In [ ]:
# Images with Defects

train_images_have_defects_set = set(train_df['ImageId'])
len(train_images_have_defects_set)

In [ ]:
# Using the Set with Defect, we can Derive the Non-Defect Set.

train_images_no_defects_set = train_images_set - train_images_have_defects_set
no_unique_train_images_no_defects = len(train_images_no_defects_set)

print("Number of Images in train_df that have Defects >>>", len(train_images_have_defects_set))
print("Therefore, Number of Images that have No Defects >>>", str(no_unique_train_images_no_defects))

In [ ]:
# Here we Create a Containing Entries of All Images, and we Mark them as Defect = 0, Non-Defect = 1.
train_images_no_defects_df = pd.DataFrame({"ImageId" : list(train_images_no_defects_set)})
train_images_no_defects_df['allMissing'] = 1
train_images_have_defects_df = pd.DataFrame({"ImageId" : list(train_images_have_defects_set)})
train_images_have_defects_df['allMissing'] = 0
frames = [train_images_no_defects_df, train_images_have_defects_df]
train_nan_df = pd.concat(frames, ignore_index=True)

print(train_nan_df.shape)
display(train_nan_df)

In [ ]:
# Create train generator
def create_train_gen():
    return ImageDataGenerator(rescale=1/255.).flow_from_dataframe(
        train_nan_df,
        directory='../input/severstal-steel-defect-detection/train_images/',
        x_col='ImageId',
        class_mode=None,
        target_size=(256, 256),
        batch_size= 32,
        shuffle=False
    )
train_gen = create_train_gen()

In [ ]:
y_test = model.predict_generator(
    train_gen,
    steps=len(train_gen),
    verbose=1
)

In [ ]:
train_nan_df['allMissing'] = y_test

train_nan_df.to_csv('train_missing_count.csv', index=False)

## Results from Model on the Trained Dataset to be Downloaded as .csv File

In [ ]:
os.chdir(r'/kaggle/working/')
from IPython.display import FileLink

train_nan_df['allMissing'] = y_test
train_nan_df.to_csv('train_missing_count.csv', index=False)

In [ ]:
FileLink(r'train_missing_count.csv')